In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# from skimage import measure
import pyvista as pv
pv.set_jupyter_backend('static')
from tqdm import tqdm, trange
from scipy import interpolate, ndimage, fft
from scipy.special import erf
from scipy.interpolate import splrep, splev
from scipy.io import savemat
from time import time

In [2]:
from Vortex import *

In [3]:
def sample_k(k_mean,k_cov):
    return np.random.multivariate_normal(k_mean,k_cov)

#### Superpositioning the random wave ####
def rotation_matrix(axis, phi):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / np.sqrt(np.dot(axis, axis))
    a = np.cos(phi / 2.0)
    b, c, d = -axis * np.sin(phi / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def sample_wave_MO_complex(r_grid, k_mean, k_cov, n_wave = 100, kappa=1e8):
    rho = np.zeros_like(r_grid[0]).astype('complex64')
    r_grid = [r.astype(np.float32) for r in r_grid]
    for i in range(n_wave):
        k_sample = sample_k(k_mean,k_cov)

        # misorientation
        """
        https://en.wikipedia.org/wiki/Von_Mises%E2%80%93Fisher_distribution
        https://doi.org/10.1080/03610919408813161
        """
        sigma = 1e-6
        xi = np.random.rand()
        theta = np.random.rand()*2*np.pi
        W = 1+1/kappa*(np.log(xi*(1-(xi-1)/xi*np.exp(-2*kappa))))
        phi = np.arccos(W)
        axis = [np.cos(theta),np.sin(theta),0]
        R = rotation_matrix(axis,phi)
        k_sample_rot = R@k_sample

        k_dot_r = np.sum([r_grid[x]*k_sample_rot[x] for x in range(3)],axis=0)
        phi_r = np.random.rand()*2*np.pi # random phase
        rho_i = np.exp(1j*(k_dot_r + phi_r)) # cos(k_n.r + phi_n)
        rho += rho_i.astype('complex64')

    rho = np.sqrt(2/n_wave)*rho
    
    return rho


In [4]:
def visualize_defect(pl, r_grid,rho_real,sample_ordered,
                       lamellar=False,isometric=False,alpha=0,
                       lw=1,
                       color='#A0A0A0', color_back = "#303030", opacity=1, pos = "xy",
                       filename = './test_defect.png'):
    # pyvista
    # https://stackoverflow.com/questions/6030098
    grid = pv.StructuredGrid(r_grid[1], r_grid[0], r_grid[2])
    grid["vol"] = rho_real.flatten('F')
    mesh = grid.contour([alpha])

    if lamellar:
        backface_params = dict(color=color_back,
                            ambient=0.2, diffuse=0.8, specular=0.1, specular_power=10,
                            opacity=opacity
                            )
        actor = pl.add_mesh(mesh, show_scalar_bar=False, color=color,  
                    ambient=0.2, diffuse=0.8, specular=0.1, specular_power=10,
                    backface_params=backface_params, 
                    smooth_shading=True, 
                    opacity=opacity
                    )
        # bprop = actor.backface_prop
        # bprop = backface_params

    pl.add_mesh(mesh, opacity=0, show_scalar_bar=False)

    if 1:
        gridsize = r_grid[0].shape[0]
        def polyline_from_points(points):
            poly = pv.PolyData()
            poly.points = points
            the_cell = np.arange(0, len(points), dtype=np.int_)
            the_cell = np.insert(the_cell, 0, len(points))
            poly.lines = the_cell
            return poly

        for i, points in enumerate(sample_ordered):
            if len(sample_ordered[i])>=3:
                polyline = polyline_from_points(np.array(points)/gridsize*2-1)
                tangent_i = np.array(get_tangent(points))
                # theta = np.arccos(tangent_i)
                polyline["scalars"] = tangent_i[:,2]**2
                tube = polyline.tube(radius=0.01*lw)
                pl.add_mesh(tube, show_scalar_bar=False, 
                            clim=[0,1], cmap='viridis')

    if isometric:
        # camera setting
        # pl.enable_parallel_projection()
        pl.camera_position = pos
        pl.camera.reset_clipping_range()
    else:
        # camera setting
        pl.camera_position = 'yz'
        pl.camera.azimuth = -60.0
        pl.camera.elevation = 24.0
        pl.camera.reset_clipping_range()

    # light setting
    light = pv.Light()
    light.set_direction_angle(21, -55.0)
    light.attenuation_values = (0,0,2)
    pl.add_light(light)

    pl.add_bounding_box()
    # pl.show(screenshot=filename)
    # 

In [5]:
## generate isotropic randomwave
## Meshgrid
n_grid = 128
x = np.linspace(-1,1,n_grid+1)
y = np.linspace(-1,1,n_grid+1)
z = np.linspace(-1,1,n_grid+1)

r_grid = np.meshgrid(x,y,z) 

#### Wave vector distribution ####
scale = 2
k_mean_z = np.array([0,0,scale])*2*np.pi # lamellar perpendicular to z axis 

theta = np.arctan2(r_grid[0], r_grid[1]) # angle about z axis
k_dot_r = np.sum([r_grid[x]*k_mean_z[x] for x in range(3)],axis=0) 

rho = np.exp(1j*(k_dot_r/2 + theta + 0*np.pi/4)) # shift the phase

n_grid_scale = 128
scale_zoom = n_grid_scale/rho.shape[0]

def scale_rho(rho, r_grid, scale):
    rho = ndimage.zoom(rho, scale, order=1)
    r_grid = np.array([ndimage.zoom(r, scale, order=1) for r in r_grid]) 
    return rho, r_grid

rho, r_grid = scale_rho(rho, r_grid, scale_zoom)
radius = np.sqrt(np.sum([r_grid[x]**2 for x in range(2)],axis=0))

rho_real = rho.real
rho_imag = rho.imag
rho_phase = np.angle(rho)
rho_amplitude = np.absolute(rho)

# # scale the phase
# phase_scale = 1

# rho_phase = rho_phase*phase_scale
# rho = np.exp(1j*rho_phase)*rho_amplitude

# rho_real = rho.real
# rho_imag = rho.imag

# shift the phase
m_shift = -0.2
rho_real = (rho.real)**2 - np.exp(-radius**2) + m_shift
rho_imag = rho.imag
rho = rho_real + 1j*(rho_imag)
rho_phase = np.angle(rho)
rho_amplitude = np.absolute(rho)

#----------------------------------------------------------------
## find vortex lines
vortex_volume = vortex_phase(rho_phase)

x_scale = 2
### Trace defect lines
array = vortex_volume
positions_tuple_list, positions_list, adjacent_list, adjacent_id_list = get_adjacency(array)
poslist_arr = np.array(positions_tuple_list)

sample, sample_adj_list, sample_adj_dict = get_core(positions_tuple_list, adjacent_id_list, 
                                                cluster_size=3)
sample_ordered = get_ordered_points(sample, sample_adj_list)

sample_ordered_sm = []
tangent_list = []
for line in sample_ordered:
    if len(line) == 0:
        continue
    if len(line)>5:
        x = np.arange(len(line))
        x_fine = np.arange(len(line)*x_scale)/x_scale
        arr_line = np.array(line)
        ll = len(line)
        spl_list = [splrep(x,arr_line[:,i],w=np.ones(len(line)),s=ll+np.sqrt(ll*2)) for i in range(3)]
        line_sm = np.array([splev(x_fine,spl) for spl in spl_list]).T
        sample_ordered_sm.append(line_sm)

        # get line tangent
        tangent = np.array(get_tangent(line_sm))
        tangent_list.append(tangent)


In [6]:
# Visualization
pv.set_plot_theme('document')
pl = pv.Plotter(window_size=[600, 600])
# pl.enable_anti_aliasing('msaa')

visualize_defect(pl, r_grid, rho.real, sample_ordered_sm,lw=2,
    lamellar=True, color = "#A03333", color_back = "#303030", opacity=1,
    filename = 'test.png')

visualize_defect(pl, r_grid, rho.imag, sample_ordered_sm,lw=2,
    lamellar=True, color = "#3333A0", color_back = "#303030", opacity=0.25, isometric=True,
    filename = 'test.png')

filename = 'test_screw.png'
pl.show(screenshot=filename)

Widget(value='<iframe src="http://localhost:62038/index.html?ui=P_0x2aed07e6270_0&reconnect=auto" class="pyvis…

In [7]:
## generate isotropic randomwave
## Meshgrid
n_grid = 128
x = np.linspace(-1,1,n_grid+1)
y = np.linspace(-1,1,n_grid+1)
z = np.linspace(-1,1,n_grid+1)

r_grid = np.meshgrid(x,y,z) 

#### Wave vector distribution ####
scale = 2
# k_mean_z = np.array([0,0,scale])*2*np.pi # lamellar perpendicular to z axis 
k_mean_x = np.array([scale,0,0])*2*np.pi # lamellar perpendicular to z axis 

theta = np.arctan2(r_grid[0], r_grid[1]) # angle about z axis
k_dot_r = np.sum([r_grid[x]*k_mean_x[x] for x in range(3)],axis=0) 

rho = np.exp(1j*(k_dot_r/2 + theta + 0*np.pi/4)) # shift the phase

n_grid_scale = 128
scale_zoom = n_grid_scale/rho.shape[0]

def scale_rho(rho, r_grid, scale):
    rho = ndimage.zoom(rho, scale, order=1)
    r_grid = np.array([ndimage.zoom(r, scale, order=1) for r in r_grid]) 
    return rho, r_grid

rho, r_grid = scale_rho(rho, r_grid, scale_zoom)
radius = np.sqrt(np.sum([r_grid[x]**2 for x in range(2)],axis=0))

rho_real = rho.real
rho_imag = rho.imag
rho_phase = np.angle(rho)
rho_amplitude = np.absolute(rho)

# # scale the phase
# phase_scale = 1

# rho_phase = rho_phase*phase_scale
# rho = np.exp(1j*rho_phase)*rho_amplitude

# rho_real = rho.real
# rho_imag = rho.imag

# shift the phase
m_shift = -0.2
rho_real = (rho.real)**2 - np.exp(-radius**2) + m_shift
rho_imag = rho.imag
rho = rho_real + 1j*(rho_imag)
rho_phase = np.angle(rho)
rho_amplitude = np.absolute(rho)

#----------------------------------------------------------------
## find vortex lines
vortex_volume = vortex_phase(rho_phase)

x_scale = 2
### Trace defect lines
array = vortex_volume
positions_tuple_list, positions_list, adjacent_list, adjacent_id_list = get_adjacency(array)
poslist_arr = np.array(positions_tuple_list)

sample, sample_adj_list, sample_adj_dict = get_core(positions_tuple_list, adjacent_id_list, 
                                                cluster_size=3)
sample_ordered = get_ordered_points(sample, sample_adj_list)

sample_ordered_sm = []
tangent_list = []
for line in sample_ordered:
    if len(line) == 0:
        continue
    if len(line)>5:
        x = np.arange(len(line))
        x_fine = np.arange(len(line)*x_scale)/x_scale
        arr_line = np.array(line)
        ll = len(line)
        spl_list = [splrep(x,arr_line[:,i],w=np.ones(len(line)),s=ll+np.sqrt(ll*2)) for i in range(3)]
        line_sm = np.array([splev(x_fine,spl) for spl in spl_list]).T
        sample_ordered_sm.append(line_sm)

        # get line tangent
        tangent = np.array(get_tangent(line_sm))
        tangent_list.append(tangent)

In [8]:
# Visualization
pv.set_plot_theme('document')
pl = pv.Plotter(window_size=[600, 600])
# pl.enable_anti_aliasing('msaa')

visualize_defect(pl, r_grid, rho.real, sample_ordered_sm,lw=2,
    lamellar=True, color = "#A03333", color_back = "#303030", opacity=1,
    filename = 'test.png')

visualize_defect(pl, r_grid, rho.imag, sample_ordered_sm,lw=2,
    lamellar=True, color = "#3333A0", color_back = "#303030", opacity=0.25, isometric=True,
    filename = 'test.png')

filename = 'test_edge.png'
pl.show(screenshot=filename)

Widget(value='<iframe src="http://localhost:62038/index.html?ui=P_0x2aedc77bed0_1&reconnect=auto" class="pyvis…